# Deep Reinforcement Learning
In this lab we will implement and train an agent that uses deep learning to play balance the stick in `CartPole-v1`.

## Setup
----
We import useful packages: `gym`, `torch` stuff, etc..

Imports:

In [7]:
import gym
import random
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from collections import deque  # for memory
from tqdm import tqdm          # for progress bar

How the game looks (without our agent):

In [2]:
env = gym.make('CartPole-v1', render_mode='human')
for _ in tqdm(range(10)):
    state, _ = env.reset()
    done = False
    while not done:
        action = env.action_space.sample()
        next_state, reward, done, _, _ = env.step(action)
env.close()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.95it/s]


## DQN Algorithm
-------------
We train a policy that tries to maximize the discounted,
cumulative reward
$R_{t_0} = \sum_{t=t_0}^{\infty} \gamma^{t - t_0} r_t$, where
$R_{t_0}$ is *return*. The discount, $\gamma$ is the discount, between $0$ and $1$


Q-learning tries to find a function
$Q^*: State \times Action \rightarrow \mathbb{R}$, maximizes rewards:

\begin{align}\pi^*(s) = \arg\!\max_a \ Q^*(s, a)\end{align}

However, we don't know $Q^*$. So, we use neural network as a approximators, we can simply create one and train it to resemble $Q^*$.

For our training update rule, we'll use a fact that every $Q$
function for some policy obeys the Bellman equation:

\begin{align}Q^{\pi}(s, a) = r + \gamma Q^{\pi}(s', \pi(s'))\end{align}

The difference between the two sides of the equality is known as the temporal difference error, $\delta$:

\begin{align}\delta = Q(s, a) - (r + \gamma \max_a Q(s', a))\end{align}

### Model
---
Make a deep learning based policy model, that takes in a state and outputs an action.
This model will be an attribute of the Agent we make next.

In [2]:
class Model(nn.Module):
    def __init__(self, observation_size, action_size):
        super(Model, self).__init__()
        self.dense1 = nn.Linear(observation_size, 100)
        torch.nn.init.xavier_uniform_(self.dense1.weight)
        self.dense2 = nn.Linear(100, 100)
        torch.nn.init.xavier_uniform_(self.dense2.weight)
        self.dense3 = nn.Linear(100, action_size)
        torch.nn.init.xavier_uniform_(self.dense3.weight)

    def forward(self, x):
        x = self.dense1(x)
        x = F.relu(x)
        x = self.dense2(x)
        x = F.relu(x)
        x = self.dense3(x)
        return x

    def predict(self, x):
        x = torch.tensor(x)
        x = self.forward(x)
        return torch.argmax(x) 

### DQN Agent
----
We will be using experience replay memory for training our model.
An Agent's memory is as important as its model, and will be another attribute of our agent.
Other appropriate attributes are the hyperparameters (gamma, lr, etc.).
Give the agent a replay method that trains on a batch from its memory.


In [9]:

class Agent:
    def __init__(self, observation_size, action_size):
        self.observation_size=observation_size
        self.action_size = action_size
        self.criterion = nn.MSELoss()
        self.model = Model(observation_size, action_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=1e-3)
        self.N = 2000
        self.explore_rate = 0.15
        self.explore_decay = 0.99
        self.explore_min = 0.0
        self.discount_rate = 0.9
        #self.memory = Queue.queue(self.N)
        self.memory = deque([], maxlen=self.N)
        # self.memory = torch.tensor(np.array.zeros(self.N, 4)) # memory that stores N most new transitions
        # good place to store hyperparameters as attributes

    def remember(self, state, action, reward, next_state, done):
        self.memory.append([state, action, reward, next_state, done])

    def act(self, state, use_random=True):
        if random.random() < self.explore_rate and use_random:
            return random.randint(0, 1)

        action = self.model.predict(state)
        action = int(action)
        return action

    def replay(self, batch_size):

        minibatch = random.sample(self.memory, batch_size)
        self.optimizer.zero_grad()

        for i in range(batch_size):
            self.train(minibatch[i])

        self.optimizer.step()

    def train(self, sample):
        s, a, r, s1, d = sample
        s = torch.tensor(s)
        s1 = torch.tensor(s1)
        r = torch.tensor(r)

        if not d:
            v = r + self.discount_rate * float(torch.max(self.model.forward(s1)))
        else:
            v = r
        pred = self.model.forward(s)[a]
        loss = self.criterion(pred, v)
        loss.backward()

### Main Training loop
---
Make a function that takes and environment and an agent, and runs through $n$ episodes.
Remember to call that agent's replay function to learn from its past (once it has a past).


In [12]:
def train(env, agent, episodes=200, batch_size=64, plotting=False):  # train for many games
    if plotting:
        plt.ion()
        plt.show()
        plt.xlabel('Episode')
        plt.ylabel('Score')
        plt.title('Cart pole')
        x = []
        y = []

    for e in tqdm(range(episodes)):
        state, _ = env.reset()
        done = False
        total_r = 0
        iter = 0
        while not done:
            # 1. make a move in game.
            
            action = agent.act(state)

            #print(action)
            next_state, reward, done, _, _ = env.step(action)
 
            total_r += reward

            # 2. have the agent remember stuff.
            #agent.remember(state, action, reward, next_state, done)
            agent.remember(state, action, reward, next_state, done)

            # 3. update state
            state = next_state

            # 4. if we have enough experiences in out memory, learn from a batch with replay.
            if len(agent.memory) >= batch_size:
                agent.replay(batch_size)
            iter += 1

        # plotting
        if plotting:
            x.append(e)
            y.append(total_r)
            plt.plot(x, y)
            plt.draw()
            plt.pause(0.1)

### Putting it together
---
We train an agent on the environment:


In [ ]:
env = gym.make('CartPole-v1' , render_mode='human')
agent = Agent(env.observation_space.shape[0], env.action_space.n)
train(env, agent)
env.close()

## Optional (highly recommended): Atari
Adapt your agent to play an Atari game of your choice.
https://www.gymlibrary.dev/environments/atari/air_raid/